# 2.1 Unicode

## <font color="red">(a)</font>

In [61]:
chr(0)

'\x00'

## <font color="red">(b)</font>

In [62]:
print(chr(0))

 


In [63]:
l = "this is a test" + chr(0) + "string"
l

'this is a test\x00string'

## <font color="red">(c)</font>

In [64]:
print(l)

this is a test string


# 2.2 Unicode  Encoding

## <font color="red">(a)</font>

In [65]:
# UTF-8编码
"hello".encode("utf-8")  # b'hello' - ASCII字符保持原样
# 每个ASCII字符正好是1字节

# UTF-16编码
"hello".encode("utf-16-le")  # b'h\x00e\x00l\x00l\x00o\x00'
# 每个ASCII字符变为2字节，其中一半是空字节(0x00)

# UTF-32编码
"hello".encode("utf-32-le")  # b'h\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00'
# 每个ASCII字符变为4字节，其中3/4是空字节

b'h\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00'

## <font color="red">(b)</font>

原因: 它逐个字节解码UTF-8，而UTF-8字符可能是多字节的.
破坏了多字节序列.UTF-8多字节字符必须作为一个整体解码
,拆分字节会得到完全不同的字符或无效序列.
eg. "你" 的UTF-8编码是 b'\xe4\xbd\xa0'
如果逐个字节解码,会得到:
b'\xe4' -> 'ä'
b'\xbd' -> '½'
b'\xa0' -> ' '
这不是一个有效的UTF-8字符序列,因此会被解码为替换字符(U+FFFD).

In [66]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

string = "hello"
decode_utf8_bytes_to_str_wrong(string.encode("utf-8"))

'hello'

In [67]:
print("你".encode("utf-8"))
print("你".encode("utf-8").decode("utf-8"))

b'\xe4\xbd\xa0'
你


In [68]:
print(b'\xe4'.decode("utf-8"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 0: unexpected end of data

In [ ]:
string = "你"
decode_utf8_bytes_to_str_wrong(string.encode("utf-8"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 0: unexpected end of data

## <font color="red">(c)</font>

"中".encode("utf-8")  # b'\xe4\xb8\xad' 是有效的三字节序列
<br>
#但如果拆分：b'\xe4\xb8' 就只有两个字节，但缺少最后一个字节，也是无效的

# 2.4 Pre-tokenization

In [76]:
import regex as re
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
print(re.findall(PAT, "你好，这是一个测试字符串！"))
print(re.findall(PAT, "some text that i'll pre-tokenize"))

['你好', '，', '这是一个测试字符串', '！']
['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']


# 2.5 Experimenting with BPE Tokenizer Training

相比https://github.com/heng380/cs336-assignment1的改进



## 新tokenizer相比旧版本的主要改变和优化

### 1. **注释和文档改进**
- **所有注释中文化**：将英文注释全部替换为中文注释，便于中文用户理解
- **增加详细文档**：为每个函数和类添加了详细的中文文档字符串

### 2. **分块处理优化**
- **换行符规范化**：在`get_chunk`函数中增加了换行符统一处理，将`\r\n`和`\r`都转换为`\n`
- **分割标记更改**：将分块分割标记从`b"<|endoftext|>"`改为`b"\n"`，更符合实际文本分割需求

### 3. **核心算法性能优化**

#### `count_pair_frequencies`方法优化
```python
# 旧版本：每次循环都计算len(word)
for word, freq in tokens_counter.items():
    for i in range(len(word) - 1):

# 新版本：避免重复计算长度，提高性能
for word, freq in tokens_counter.items():
    word_len = len(word)  # 优化：避免重复计算长度
    for i in range(word_len - 1):
```

#### `find_max_pair`方法重写
- **旧版本**：使用`max(counts, key=lambda x: (counts[x], x))`
- **新版本**：手动实现循环比较，避免lambda函数开销，并修复了字典序选择逻辑
- **重要修复**：当频率相同时，选择字典序更大的pair（与原始代码一致）

#### `merge_tokens`方法重大优化
**主要改进：**
- **原地更新**：不再创建新的Counter对象，而是原地更新现有的tokens_counter
- **pair_counts维护**：新增pair_counts参数，在合并时实时更新字节对频率
- **批量操作**：使用tokens_to_remove和tokens_to_add列表进行批量更新
- **性能提升**：避免重复计算pair_counts，显著提高训练速度

### 4. **训练过程优化**

#### `train_BPE`方法改进
- **进程数限制**：`num_processes = min(multiprocessing.cpu_count(), 8)`，避免过度并行化
- **chunksize优化**：使用`chunksize=max(1, len(chunks) // num_processes)`提高并行效率
- **pair_counts单次初始化**：在训练循环外初始化pair_counts，避免重复计算
- **进度条优化**：提前计算目标词汇量，进度条显示更准确

### 5. **Tokenizer类编码优化**
- **移除tqdm进度条**：在`encode`方法中移除了tqdm进度条，提高编码性能
- **保持功能完整性**：所有核心功能保持不变

### 6. **代码结构和可读性改进**
- **类和方法注释**：为所有类和方法添加了详细的中文注释
- **变量命名更清晰**：如`tokens_to_remove`、`tokens_to_add`等变量名更直观
- **代码逻辑分组**：使用中文注释对代码进行逻辑分组

## 性能提升总结

1. **训练速度提升**：通过pair_counts的单次初始化和原地更新，显著减少重复计算
2. **内存效率提升**：避免创建不必要的Counter对象，减少内存分配
3. **并行效率优化**：合理的进程数限制和chunksize设置
4. **算法复杂度优化**：减少重复的长度计算和字典操作

        

# 对于因果掩码的理解